In [1]:
!pip install -r requirements.txt

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torchvision-0.18.0-cp39-cp39-win_amd64.whl.metadata (6.6 kB)
  Using cached PyYAML-5.1.2.tar.gz (265 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached scikit-image-0.15.0.tar.gz (32.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached scipy-1.3.1.tar.gz (23.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [344 lines of output]
      Ignoring numpy: markers 'python_version == "3.5" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.5" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.7" and platform_system == "AIX"' don't match your environment
        Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
        Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
        Using cached Cython-3.0.10-cp39-cp39-win_amd64.whl.metadata (3.2 kB)
        Using cached numpy-1.14.5.zip (4.9 MB)
  

#### Load Data

#### Load Models

#### Evaluate Models 
